In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np 
import pandas as pd 

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras

print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.13.1


In [3]:
data = pd.read_csv("all.csv")

In [4]:
data['category'][0]

'confidentialR'

In [5]:
data['text'][0]

'CVE-2019-9999 ** RESERVED **  has been reserved by an organization or individual that will use it when announcing a new security problem. When the   has been publicized, the details for  will be provided.\n\n'

In [6]:
max_words = 10000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words)

In [7]:
tokenize.fit_on_texts(data['text']) # fit tokenizer to our training text data

In [8]:
# x_train = tokenize.texts_to_matrix(data['text'])

In [9]:
tokenize.word_index

{'•': 1,
 'the': 2,
 'and': 3,
 'to': 4,
 'of': 5,
 'a': 6,
 'server': 7,
 'support': 8,
 'engineer': 9,
 'line': 10,
 'microsoft': 11,
 'for': 12,
 'will': 13,
 'in': 14,
 'security': 15,
 'as': 16,
 'sc': 17,
 'on': 18,
 'is': 19,
 '3rd': 20,
 'cleared': 21,
 'veritas': 22,
 'windows': 23,
 'fujitsu': 24,
 'or': 25,
 'by': 26,
 'its': 27,
 'team': 28,
 'skills': 29,
 'be': 30,
 'an': 31,
 'with': 32,
 'this': 33,
 'all': 34,
 '\r': 35,
 'ms': 36,
 'role': 37,
 'information': 38,
 'services': 39,
 'desktop': 40,
 'applications': 41,
 'technologies': 42,
 'it': 43,
 'are': 44,
 'level': 45,
 'management': 46,
 'provide': 47,
 'system': 48,
 'control': 49,
 'solutions': 50,
 'active': 51,
 'systems': 52,
 'storage': 53,
 'clearance': 54,
 'salary': 55,
 'sharepoint': 56,
 'servers': 57,
 'remote': 58,
 'government': 59,
 'group': 60,
 'have': 61,
 'part': 62,
 'issues': 63,
 'within': 64,
 'people': 65,
 'including': 66,
 'order': 67,
 'through': 68,
 'working': 69,
 'work': 70,
 'would

In [10]:
# tokenize.fit_on_sequences(data['text'])

In [11]:
# tokenize.texts_to_sequences

In [12]:
x_data = tokenize.texts_to_sequences(data['text'])

In [13]:
print(x_data[0])

[398, 399, 400, 120, 118, 117, 120, 26, 31, 394, 25, 396, 114, 13, 395, 43, 119, 401, 6, 392, 15, 116, 119, 2, 118, 117, 402, 2, 397, 12, 13, 30, 115]


In [14]:
encoder = LabelEncoder()
encoder.fit(data['category']) 
y_data = encoder.transform(data['category'])

In [15]:
for i in range(len(x_data)):
    x_data[i] = [x+3 for x in x_data[i]]

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
train_data, test_data, train_labels, test_labels = train_test_split(x_data, y_data, test_size=0.33, random_state=42)

In [18]:
print(f"Training entries {len(train_data)}. Labels: {len(train_labels)}")

Training entries 2320. Labels: 2320


In [19]:
print(f"Testing entries {len(test_data)}. Labels: {len(test_labels)}")

Testing entries 1144. Labels: 1144


In [20]:
len(train_data[0])

815

In [21]:
# tokenize.sequences_to_texts(sequences)

In [22]:
print(train_labels[1])

2


In [23]:
word_index = tokenize.word_index

In [24]:
word_index = {k : (v+3) for k, v in word_index.items()}

In [25]:
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

In [26]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [27]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, "?") for i in text])

In [28]:
print(decode_review(train_data[0]))

3rd line support engineer technical lead sc cleared job title 3rd line support engineer security clearance to sc level required location bracknell salary negotiable dependant on experience skills required veritas storage foundation veritas volume replicator windows server windows xp desktop active directory dns dhcp dfs group policy objects server failure debugging microsoft sharepoint server windows about fujitsu fujitsu uk and ireland is a leading it systems services and products company employing people with an annual revenue of billion its business is in enabling its customers to realise their objectives by exploiting information technology through its integrated product and service portfolio this includes consulting applications systems integration managed services and product for customers in the private and public sectors including retail financial services telecoms and government defence and consumer sectors the role fujitsu are currently looking for a security cleared 3rd line

In [29]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding="post",
                                                        maxlen=1024)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding="post",
                                                       maxlen=1024)

In [30]:
print(len(train_data[0]), len(train_data[1]))

print(train_data[0])

vocab_size = 10000

1024 1024
[23 13 11 ...  0  0  0]


In [31]:
from keras.utils import to_categorical
org_test_labels = test_labels
org_train_labels = train_labels
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Using TensorFlow backend.


In [32]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(3, activation=tf.nn.softmax))

Instructions for updating:
Colocations handled automatically by placer.


In [33]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["acc"])

In [34]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]


In [35]:
history = model.fit(train_data, train_labels,
                    batch_size=512,
                    epochs=1000,
                    verbose=1,
                    validation_split=0.1)

Train on 2088 samples, validate on 232 samples
Instructions for updating:
Use tf.cast instead.


InternalError: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 8366784512

In [ ]:
# model.fit(partial_x_train,partial_y_train,epochs=40,batch_size=512,verbose=1)

In [ ]:
results = model.evaluate(test_data, test_labels)

print(results)

In [ ]:
index = 1100
test_review = test_data[index]
predict = model.predict_classes([test_review])
print("Text: ")
print(decode_review(test_review))
print("Prediction: " + str(predict[0]))
print("Prediction class name: " + str(encoder.inverse_transform([predict[0],0])[0]))
print("Actual: "+str(org_test_labels[index]))
print("Actual class name: " + str(encoder.inverse_transform([org_test_labels[index],0])[0]))

In [ ]:
max_words = 10000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words)


In [ ]:
tokenize.fit_on_texts(data['text'])

In [ ]:
input_text = ['3rd line support engineer technical lead sc cleared job title 3rd line support engineer security clearance to sc level required location bracknell salary negotiable dependant on experience skills required veritas storage foundation veritas volume replicator windows server windows xp desktop active directory ','Do Not Remove this text']
input_seq = tokenize.texts_to_sequences(input_text)

input_seq[0] = [x+3 for x in input_seq[0]]
# print(type(input_seq[0][0]))
input_seq = keras.preprocessing.sequence.pad_sequences(input_seq,
                                                       value=word_index["<PAD>"],
                                                       padding="post",
                                                       truncating='post',
                                                       maxlen=10)


print(decode_review(input_seq[0]))
predict = model.predict_classes([input_seq])
print("Prediction: " + str(predict[0]))
print("Prediction class name: " + str(encoder.inverse_transform([predict[0],0])[0]))

In [ ]:
# model.save("model.h5")
# print("Saved model to disk")

In [ ]:
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model('model.h5')
model.summary()

In [ ]:
results = model.evaluate(test_data, test_labels)